In [1]:
import numpy as np

In [2]:
import mne

In [3]:
import pandas as pd

In [4]:
import json

In [5]:
%matplotlib
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [6]:
%load_ext autoreload

%autoreload 2
import tools

In [7]:
import numpy as np

In [8]:
import itertools

In [10]:
EMPTY_ACTION = -1

In [11]:
import itertools

In [12]:
import eeg_events

In [42]:
class OfflineJoluEngagementEnvironment():
    filename = None
    iterator = None
    event = None
    
    states = ["".join(t) for t in itertools.product(*[[str(i) for i in range(0, 6)]] * 3)]
    actions = {
        0: "o",
        1: "x",
        2: "n",
        3: "*",
        4: "u",
        5: "-",
        -1: "$"
    }
    
    def __init__(self, filename):
        self.filename = filename
    
    def reset(self):
        self.iterator = eeg_events.iterate_events(self.filename)
        self.event = next(self.iterator)
        return self.states[0]
    
    def sample(self):
        event = self.event[1]
        if not event.get("collect"):
            return self.actions[event["response"]['result']["reels"]["view"][0][0]]
        else:
            return self.actions[EMPTY_ACTION]
        
    def get_state(self):
        event_tuple = self.event
        event = event_tuple[1]
        state_array = event["response"]['result']["state"]["towerStages"]
        state = "".join([str(i) for i in state_array])
        return state
    
    def get_reward(self):
        event_tuple = self.event
        event_data = event_tuple[2]
        assert event_data.shape[0] > 0
        return np.median(event_data)
        
    def step(self, action):
        event = self.event
        
        next_state = self.get_state()
        reward = self.get_reward()
        
        self.event = next(self.iterator, None)
        
        done = not self.event
        
        return next_state, reward, done, None

In [43]:
env = OfflineJoluEngagementEnvironment('test1')

In [44]:
env.reset()

Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.


'000'

In [45]:
done = False
while not done:
    print(env.sample())
    state_tuple = env.step('')
    _, _, done, _ = state_tuple
    print(state_tuple)

u
('001', 5.163911581039429, False, None)
u
('002', 3.6251907348632812, False, None)
o
('113', 3.287583887577057, False, None)
$
('000', 3.373575985431671, False, None)
-
('000', 3.4239096641540527, False, None)
-
('000', 3.104528248310089, False, None)
u
('001', 3.057699680328369, False, None)
-
('001', 3.2693721055984497, False, None)
-
('001', 3.0806952118873596, False, None)
-
('001', 3.011557877063751, False, None)
-
('001', 3.154055893421173, False, None)
-
('001', 3.3953155875205994, False, None)
*
('011', 3.498696982860565, False, None)
-
('011', 3.7934980988502502, False, None)
*
('021', 3.8866488933563232, False, None)
$
('000', 3.7224859595298767, False, None)
u
('001', 3.0336636304855347, False, None)
-
('001', 3.1165533661842346, False, None)
-
('001', 3.2759527564048767, False, None)
*
('011', 3.7076988220214844, False, None)
-
('011', 3.447341799736023, False, None)
-
('011', 2.9384170174598694, False, None)
-
('011', 3.1007161736488342, False, None)
*
('021', 3.74367421

### SARSA

In [47]:
env = OfflineJoluEngagementEnvironment('test1')

In [48]:
alpha = 0.15
gamma = 0.99

In [49]:
Q = {}

In [50]:
states_number = {}

In [55]:
from tqdm import tqdm_notebook

In [56]:
for i in tqdm_notebook(range(0, 100)):
    r = 0
    state = env.reset()
    action = env.sample()
    while True:
        nextstate, reward, done, _ = env.step(action)
        if done:
            break
        nextaction = env.sample()
#         print(state, action, nextstate, nextaction)
        if (state, action) not in Q:
            Q[(state, action)] = 0
            states_number[(state, action)] = []
        if (nextstate, nextaction) not in Q:
            Q[(nextstate, nextaction)] = 0
            states_number[(nextstate, nextaction)] = []
        Q[(state, action)] += alpha * (reward + gamma * Q[(nextstate, nextaction)] - Q[(state, action)])
        states_number[(state, action)].append(reward)
        action = nextaction
        state = nextstate
        r += reward

/home/anatoli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/e

Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000

Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test1.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file /home/anatoli/Документы/sk

In [57]:
symbols_engagement = list(filter(lambda x: x[-1] != 0, sorted(Q.items(), key=lambda x: x[1])))

In [58]:
def calculate_reward(state):
    bells = int(state[0])
    stars = int(state[1])
    horsehoes = int(state[2])
    return joker_rewards["bell"][bells] + joker_rewards["star"][stars] + joker_rewards["horsehoe"][horsehoes]

In [59]:
joker_rewards = {
    "bell": [0, 10, 40, 100, 200, 350],
    "star": [0, 3, 8, 16, 29, 50],
    "horsehoe": [0, 1, 2, 4, 7, 10],
    "biggest": 500
}

In [60]:
symbols_engagement = [(a, calculate_reward(a), len(states_number[(a, b)]), b, c) for (a, b), c in symbols_engagement]

In [76]:
sorted(symbols_engagement, key=lambda x: x[-1])

[('001', 1, 102, 'n', 210.95083814309703),
 ('000', 0, 102, 'n', 211.19844871021422),
 ('001', 1, 102, 'o', 218.3051727210061),
 ('101', 11, 102, '-', 219.54100916900853),
 ('002', 2, 102, 'o', 219.8729839377829),
 ('100', 10, 102, 'u', 220.42248801385935),
 ('010', 3, 102, '*', 220.84209300253852),
 ('003', 4, 102, 'n', 222.08377823391066),
 ('103', 14, 102, '$', 226.68236685141167),
 ('101', 11, 102, 'u', 227.30829581531344),
 ('112', 15, 102, '$', 228.16932688662098),
 ('002', 2, 204, 'n', 228.69975406734213),
 ('113', 17, 102, '$', 228.81298888645458),
 ('020', 8, 102, 'u', 228.82972572963044),
 ('102', 12, 204, '-', 229.21723440761585),
 ('000', 0, 102, 'x', 229.4482860913178),
 ('003', 4, 102, '$', 229.62710902888594),
 ('101', 11, 102, '$', 229.66959305389514),
 ('021', 9, 102, 'u', 229.85002278905534),
 ('003', 4, 510, '-', 229.95051693105734),
 ('013', 7, 102, 'x', 231.36756507389438),
 ('002', 2, 714, 'u', 231.63405818795917),
 ('003', 4, 204, '*', 232.53953095237),
 ('012', 

Пример - состояние 013. Вовлеченность существенно больше, если игрок находится в состоянии 0. Что это за состояние?

Как меняются приоритеты игрока в состояниях 0 и 2

In [65]:
def increase_state(s, i):
    return s[:i] + chr(ord(s[i]) + 1) + s[i+1:]

In [66]:
def get_transition(s, a):
    if a == '-':
        return s
    if a in ['$', 'x']:
        return '000'
    if a == 'u':
        return increase_state(s, 2)
    if a == 'n':
        return increase_state(s, 0)
    if a == '*':
        return increase_state(s, 1)
    if a == 'o':
        ns = increase_state(s, 0)
        ns = increase_state(ns, 1)
        ns = increase_state(ns, 2)
        return ns

In [67]:
get_transition("000", "o")

'111'

In [68]:
rewards = {}

In [69]:
for key, value in list(Q.items()):
    if not value:
        del Q[key]

In [70]:
V = {} # V - minimum value
for (s, a), value in Q.items():
    if s not in V:
        V[s] = max(Q.values())
    if value < V[s]:
        V[s] = value

In [71]:
for (s, a), value in Q.items():
    ns = get_transition(s, a)
    if (s, a) not in rewards:
        rewards[(s, a)] = 0
    rewards[(s, a)] = Q[(s, a)] - V[ns] / gamma

In [74]:
sorted(rewards.items(), key=lambda x: x[1])

[(('001', 'o'), -12.168894841237318),
 (('000', 'n'), -11.450529081562905),
 (('002', 'o'), -11.251247260656072),
 (('001', 'n'), -10.807756977113598),
 (('010', '*'), -10.299044098098278),
 (('021', 'u'), -9.959742724566041),
 (('003', 'n'), -6.88830949478799),
 (('013', 'u'), -4.310924615667318),
 (('101', 'u'), -4.224264192379337),
 (('003', 'u'), -4.01559658620107),
 (('020', 'u'), -3.3420144613345713),
 (('012', '*'), -3.0393680187017935),
 (('002', 'n'), -2.832805940350653),
 (('102', '-'), -2.3153256000769318),
 (('101', '-'), -2.2175859512020963),
 (('100', 'u'), -1.3361071063512782),
 (('003', '*'), -1.1650802333819001),
 (('013', '-'), -0.9072248322685823),
 (('011', '-'), -0.2862365742921895),
 (('001', '*'), 0.013755930803455385),
 (('010', 'u'), 0.37974177425488165),
 (('011', '*'), 0.928651867161534),
 (('012', 'u'), 1.1689789396531864),
 (('011', 'u'), 1.3960210530646577),
 (('002', '*'), 1.5113081199201304),
 (('012', '-'), 1.9897726126341126),
 (('003', '-'), 5.6234682